In [1]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")


# Load data

In [2]:
dataloc = '/LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECTS/2021_IgASeq/tables/'
zenodoloc = '/home/mattolm/user_data/Programs/MIGSeq_code/zenodo/'

In [3]:
# IgA binding data (IgA_frac); from supplemental table S1
Idb = pd.read_csv(dataloc + 'SupplementalTable_S1_v1.csv')
Idb = Idb[['Tube_ID', 'sample', 'IgA_frac', 'pass_sorting_QC', 'pass_SeqDepth_QC']].drop_duplicates()
Idb = Idb[(Idb['pass_SeqDepth_QC'] == True) & (Idb['pass_sorting_QC'] == True)]
display(Idb.head())

# Raw genome abundance data; from Zenodo
Gdb = pd.read_csv(zenodoloc + 'IgASeq_genomeDetection_v4.csv')
Gdb = Gdb[Gdb['sample'].isin(set(Idb['sample']))]
Gdb.head()

assert set(Idb['sample']) == set(Gdb['sample'])


,Tube_ID,sample,IgA_frac,pass_sorting_QC,pass_SeqDepth_QC
2,1022,OS_62,0.4660,True,True
3,1022,OS_24,0.1353,True,True
4,1026,OS_64,0.3230,True,True
5,1026,OS_26,0.0670,True,True
6,1036,OS_54,0.5630,True,True


# Prepare to calculate IgA+ probability score

In [4]:
# Make a base table
name = 'positive'
db1 = Gdb[Gdb['IgA'] == name][['genome', 'Tube_ID', 'breadth', 'rel_ab', 'coverage', 'sample']].rename(columns={'breadth':name + '_breadth', 'rel_ab':name + '_rel_ab', 'coverage':name + '_coverage', 'sample':name + '_sample'})

name = 'unsorted'
db2 = Gdb[Gdb['IgA'] == "resuspended_unsorted"][['genome', 'Tube_ID', 'breadth', 'rel_ab', 'coverage', 'sample']].rename(columns={'breadth':name + '_breadth', 'rel_ab':name + '_rel_ab', 'coverage':name + '_coverage', 'sample':name + '_sample'})

IGdb = pd.merge(db1, db2, on=['genome', 'Tube_ID'], how='outer')
IGdb['Tube_ID'].value_counts()

assert len(IGdb[IGdb.duplicated(subset=['genome', 'Tube_ID'], keep=False)].sort_values(['genome', 'Tube_ID'])) == 0

# Figure out which NaNs should be 0s:
for name in ['positive', 'unsorted']:
    # Tube ID to sample
    t2i = IGdb.dropna(subset=[name + '_sample']).set_index('Tube_ID')[name + '_sample'].to_dict()
    
    # Backfill to remove the NaNs
    IGdb[name + '_sample'] = IGdb['Tube_ID'].map(t2i)

    # Backfill to remove the 0s
    for c in ['breadth', 'coverage', 'rel_ab']:
        IGdb[name + '_' + c] = [0 if ((s == s) & (c_val != c_val)) else c_val for c_val, s in zip(IGdb[name + '_' + c], IGdb[name + '_sample'])]
        
# Add IgA binding
IGdb = pd.merge(IGdb, Idb[['Tube_ID', 'IgA_frac', 'pass_sorting_QC']], how='left')
assert set(IGdb['unsorted_sample']).union(set(IGdb['positive_sample'])) == set(Gdb['sample'])
IGdb


,genome,Tube_ID,positive_breadth,positive_rel_ab,positive_coverage,positive_sample,unsorted_breadth,unsorted_rel_ab,unsorted_coverage,unsorted_sample,IgA_frac,pass_sorting_QC
0,GUT_GENOME000139.fna.fa,7028,0.552788,0.000369,1.121264,OS_61,0.000000,0.000000,0.000000,OS_23,0.8070,True
1,GUT_GENOME000139.fna.fa,7028,0.552788,0.000369,1.121264,OS_61,0.000000,0.000000,0.000000,OS_23,0.0767,True
2,GUT_GENOME000205.fna.fa,7028,0.267296,0.000627,1.724991,OS_61,0.119370,0.000303,1.145421,OS_23,0.8070,True
3,GUT_GENOME000205.fna.fa,7028,0.267296,0.000627,1.724991,OS_61,0.119370,0.000303,1.145421,OS_23,0.0767,True
4,GUT_GENOME000221.fna.fa,7028,0.020986,0.000445,0.920143,OS_61,0.068223,0.001864,5.255976,OS_23,0.8070,True
...,...,...,...,...,...,...,...,...,...,...,...,...
20443,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.919023,0.000955,4.243006,OS_5,0.0860,True
20444,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.497457,0.000286,0.865073,OS_5,0.6760,True
20445,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.497457,0.000286,0.865073,OS_5,0.0860,True
20446,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.802729,0.000426,2.240798,OS_5,0.6760,True


## Calulate IgA binding metrics

In [5]:
def calc_iga_prob(row):
    if row['unsorted_rel_ab'] == 0:
        if row['positive_rel_ab'] > 0:
            return 1e6
        else:
            return np.nan
    return (row['positive_rel_ab'] * row['IgA_frac'] / row['unsorted_rel_ab'])

IGdb['IgA_prob'] = IGdb.apply(calc_iga_prob, axis=1)

IGdb['semi_robustly_detected'] = [(p >= 0.5) | (u >= 0.5) for p, u in zip(IGdb['positive_breadth'], IGdb['unsorted_breadth'])]
IGdb['robustly_detected'] = [(p >= 0.5) & (u >= 0.5) for p, u in zip(IGdb['positive_breadth'], IGdb['unsorted_breadth'])]


# Correct for sequencing depth


In [6]:
THRESH = 12
gs = 4000000
rel_thresh = gs/(THRESH*1e9)

# Make a filtered dataset that only include genomes that pass threse thresholds
FIGdb = IGdb.copy()
FIGdb = FIGdb[(FIGdb['positive_rel_ab'] >= (rel_thresh)) | (FIGdb['unsorted_rel_ab'] >= rel_thresh)]
FIGdb = FIGdb[(FIGdb['positive_breadth'] >= 0.5) | (FIGdb['unsorted_breadth'] >= 0.5)]

print(f"Analyzing {len(FIGdb['Tube_ID'].unique())} samples")


Analyzing 30 samples


In [7]:
rel_thresh

0.0003333333333333333

# Set binding classes

In [9]:
# Classify binding classes
def calc_class(iga_norm):
    if iga_norm < 0.005:
        return 'uncoated'
    elif iga_norm < 1:
        return 'low'
    else:
        assert iga_norm >= 1, iga_norm
        return 'high'
    
FIGdb['IgA_binding_class'] = [calc_class(x) for x in FIGdb['IgA_prob']]
FIGdb

,genome,Tube_ID,positive_breadth,positive_rel_ab,positive_coverage,positive_sample,unsorted_breadth,unsorted_rel_ab,unsorted_coverage,unsorted_sample,IgA_frac,pass_sorting_QC,IgA_prob,semi_robustly_detected,robustly_detected,IgA_binding_class
0,GUT_GENOME000139.fna.fa,7028,0.552788,0.000369,1.121264,OS_61,0.000000,0.000000,0.000000,OS_23,0.8070,True,1000000.000000,True,False,high
1,GUT_GENOME000139.fna.fa,7028,0.552788,0.000369,1.121264,OS_61,0.000000,0.000000,0.000000,OS_23,0.0767,True,1000000.000000,True,False,high
14,GUT_GENOME000767.fna.fa,7028,0.826847,0.000752,2.513575,OS_61,0.000000,0.000000,0.000000,OS_23,0.8070,True,1000000.000000,True,False,high
15,GUT_GENOME000767.fna.fa,7028,0.826847,0.000752,2.513575,OS_61,0.000000,0.000000,0.000000,OS_23,0.0767,True,1000000.000000,True,False,high
58,GUT_GENOME094675.fna.fa,7028,0.917889,0.001756,6.808159,OS_61,0.922662,0.002606,13.991979,OS_23,0.8070,True,0.543689,True,True,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20439,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.918784,0.000405,3.630133,OS_5,0.0860,True,0.000000,True,False,uncoated
20442,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.919023,0.000955,4.243006,OS_5,0.6760,True,0.000000,True,False,uncoated
20443,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.919023,0.000955,4.243006,OS_5,0.0860,True,0.000000,True,False,uncoated
20446,METABAT215_SUBJECTMAPPING_SCAFFOLDS_min1500_CO...,8038,0.000000,0.000000,0.000000,OS_43,0.802729,0.000426,2.240798,OS_5,0.6760,True,0.000000,True,False,uncoated
